# Google Images Web Crawler

In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import random
import time
import re
import os
import base64

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
import wget

### Setup the dogs breeds

In [ ]:
final_breeds = []

path = os.getcwd()
my_file = open(os.path.join(path ,"Breeds_File.txt"), "r")
content = my_file.read()
content_list = content.split("\n")
my_file.close()

for breed in content_list:
        word = breed.replace("_", " ").replace("-", " ").replace(" ", "")
        word= word[:1].upper() + word[1:]
        final_breeds.append(word)
        
print(final_breeds)

## Creating The Selenium Environment

In [ ]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('headless')
#driver = webdriver.Chrome(service=s, options=chrome_options)
s = Service('C:/Users/Asus/Documents/WebDriver/chromedriver.exe')
driver = webdriver.Chrome(service=s)
driver.get('https://www.google.com/')
time.sleep(5)

In [ ]:
search_URL = "https://www.google.com/search?tbm=isch&q="

### Functions

In [ ]:
def scroll_down(driver):
    img_urls = []
    try:
        # Get scroll height.
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            images = driver.find_elements(By.TAG_NAME,'img')
            time.sleep(random.randint(2,4))
            images = [image.get_attribute('src') for image in images]
            img_urls.extend(images)
            #Remove Duplicates Of Img URLs
            img_urls = list( dict.fromkeys(img_urls))
            images.clear()
            print("Images: %s" % (len(img_urls)))
            # Scroll down to the bottom.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait to load the page.
            time.sleep(random.randint(3,6))
            # Calculate new scroll height and compare with last scroll height.
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                time.sleep(random.randint(5,15))
                try:
                    load_more_clickable= WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[value='Show more results']"))).click()
                except:
                    break
            last_height = new_height
            
    except ElementNotInteractableException:
        raise Exception("Exception -> end of page")
    
    return img_urls
        
        
def download_image(url, folder_name, num):
    # write image to file
    reponse = requests.get(url)
    if reponse.status_code==200:
        path = os.getcwd()
        path = os.path.join(path, folder_name)
        with open(os.path.join(path, folder_name + str(num)+".jpg"), 'wb') as file:
            file.write(reponse.content)
            
def create_folder(query):
    if not os.path.exists(os.getcwd() + '/' + query):
        path = os.getcwd()
        path = os.path.join(path, query)
        os.mkdir(path)
        print(query + " Folder Created")
    else:
        print(query + " Folder Already Exist")
        
def save_images(images, query):
    counter = 0
    path = os.getcwd()
    path = os.path.join(path, query)
    for img in images:
        save_as = os.path.join(path ,query + str(counter) +'.jpg')
        wget.download(img, save_as)
        counter += 1
    
    print("%s Images Saved! | Total of: %s Images" %(query,counter))
    

def decode_b64(img_data, num, query):
    path = os.getcwd()
    path = os.path.join(path, query)
    
    # Separate the metadata from the image data
    head, data = img_data.split(',', 1)

    # Get the file extension (gif, jpeg, png)
    file_ext = head.split(';')[0].split('/')[1]

    # Decode the image data
    plain_data = base64.b64decode(data)
    
    #create full image path
    save_as = os.path.join(path ,query + str(num) +'.' + file_ext)
    # Write the image to a file
    with open(save_as, 'wb') as f:
        f.write(plain_data)

In [ ]:
wait = WebDriverWait(driver, 10)

for breed in final_breeds:
    
    print("Search images for: " + breed)
    
    img_urls_b64 = []
    img_urls = []
    query = breed.replace(" ", "")
    page = driver.get(search_URL + query)

    img_urls_b64 = scroll_down(driver)
    print("Images: %s" % (len(img_urls_b64)))
    
    #create path if dosent exsist
    create_folder(query)
    
    #Convert data-uri's to img files
    count = 0
    for img_b64 in img_urls_b64:
        count += 1
        decode_b64(img_b64, count, query)

In [ ]:
wait = WebDriverWait(driver, 10)

for breed in final_breeds:
    
    print("Search images for: " + breed)
    
    img_urls = []
    query = breed.replace(" ", "")
    page = driver.get(search_URL + query)

    img_urls = scroll_down(driver)
    
    page_html = driver.page_source
    pageSoup = bs(page_html, 'html.parser')
    containers = pageSoup.findAll('div', {'class':"isv-r PNCib MSM1fd BUooTd"} )

    print("Number of images for %s: %s"%(query,len(containers)))  
    
    len_containers = len(containers)
    create_folder(query)
    
    for i in range(1, len_containers+1):
        if i % 25 == 0:
            continue

        xPath = """//*[@id="islrg"]/div[1]/div[%s]"""%(i)

        previewImageXPath = """//*[@id="islrg"]/div[1]/div[%s]/a[1]/div[1]/img"""%(i)
        
        try:
            previewImageElement = driver.find_element(By.XPATH, previewImageXPath)
        except NoSuchElementException:
            pass
        
        previewImageURL = previewImageElement.get_attribute("src")
            
        driver.find_element(By.XPATH, xPath).click()
        timeStarted = time.time()

        while True:
            imageElement = driver.find_element(By.XPATH, """//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div[1]/a/img""")
            imageURL= imageElement.get_attribute('src')

            if imageURL != previewImageURL:
                #print("actual URL", imageURL)
                break

            else:
                #making a timeout if the full res image can't be loaded
                currentTime = time.time()

                if currentTime - timeStarted > 10:
                    print("Timeout! Will download a lower resolution image and move onto the next one")
                    break


        #Downloading image
        try:
            download_image(imageURL, query, i)
            print("Downloaded element %s out of %s total. URL: %s" % (i, len_containers + 1, imageURL))
        except:
            print("Couldn't download an image %s, continuing downloading the next one"%(i))
    
    time.sleep(random.randint(20,45))

### In this project all the data came from scrapping and the crawling is used for educational purpose only.